In [30]:
import ee
import geemap
import random

In [2]:
L8bands = ['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

In [3]:
Map = geemap.Map()
# Map

In [4]:
def maskL8sr(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    # Return the masked image, scaled to TOA reflectance, without the QA bands.
    return image.updateMask(mask).divide(10000).select(L8bands).copyProperties(image, ["system:time_start"])

In [5]:
# Creating an add variable function for Landsat 8 index calculation.
def compute_indices(image):
    NDVI = image.expression('(B5-B4)/(B5+B4)', 
                            {'B4': image.select('B4'),
                             'B5': image.select('B5')}).rename('ndvi')
    MNDWI = image.expression('(B3-B6) / (B3+B6)',
                             {'B3': image.select('B3'),
                              'B6': image.select('B6')}).rename('mndwi')
    EVI = image.expression('2.5 * ((B5 - B4)/(B5 +(6*B4)-(7.5*B2)+1))',
                           {'B2': image.select('B2'),
                            'B4': image.select('B4'),
                            'B5': image.select('B5')}).rename('evi')
    LSWI = image.expression('(B5-B6)/(B5+B6)', 
                            {'B5': image.select('B5'),
                             'B6': image.select('B6')}).rename('lswi')
    return image.addBands(NDVI).addBands(MNDWI).addBands(EVI).addBands(LSWI)

In [6]:
point = ee.Geometry.Point([106.4943, 20.4858])

image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(point) \
    .filterDate('2020-01-01', '2020-12-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
# Map

In [7]:
# Add the boundary of Thai Binh province
# red_river_delta_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Provinces_Polygon/Provinces_Poly.shp'
red_river_delta_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Provinces_PolygonN/Provinces_Poly.shp'
red_river_delta = geemap.shp_to_ee(red_river_delta_shp)

In [10]:
Map.addLayer(red_river_delta, {},"TB+ND+HP provinces")
Map.centerObject(red_river_delta, 9)
Map

Map(bottom=14752.0, center=[20.50075230416886, 106.47941624308001], controls=(WidgetControl(options=['position…

In [11]:
image_clip = image.clip(red_river_delta)
# Map.centerObject(point, 8)
# Map.addLayer(image_clip, vis_params, "Landsat-8")
# Map

In [12]:
img_compute = compute_indices(image_clip)

In [13]:
image_select = img_compute.select('ndvi', 'mndwi', 'evi', 'lswi')

In [14]:
# Add training markers from shape files
agriculture_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers 20201205/Training/Agriculture_Training.shp'
evergreen_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers 20201205/Training/Evergreen_Training.shp'
mangrove_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers 20201205/Training/Mangrove.shp'
others_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers 20201205/Training/Other.shp'
residence_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers 20201205/Training/Residental area_Training.shp'
shrimp_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers 20201205/Training/Shrimp.shp'
water_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers 20201205/Training/Water_Training.shp'

# agriculture_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/agriculture_markers.shp'
# evergreen_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/evergreen_markers.shp'
# mangrove_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/mangrove_markers.shp'
# others_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/other_markers.shp'
# residence_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/residence_markers.shp'
# shrimp_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/shrimp_markers.shp'
# water_shp = 'D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/Markers/water_markers.shp'

# Shape files to ee objects
tr_agriculture = geemap.shp_to_ee(agriculture_shp)
tr_evergreen = geemap.shp_to_ee(evergreen_shp)
tr_mangrove = geemap.shp_to_ee(mangrove_shp)
tr_others = geemap.shp_to_ee(others_shp)
tr_residence = geemap.shp_to_ee(residence_shp)
tr_shrimp = geemap.shp_to_ee(shrimp_shp)
tr_water = geemap.shp_to_ee(water_shp)
# Make markers collections
tr_classNames = tr_agriculture.merge(tr_evergreen).merge(tr_mangrove). \
                               merge(tr_others).merge(tr_residence). \
                               merge(tr_shrimp).merge(tr_water)
# Show the markers on map
Map.addLayer(tr_classNames, {}, 'Markers')
Map

Map(bottom=58182.0, center=[20.50075230416886, 106.47941624308001], controls=(WidgetControl(options=['position…

In [15]:
# Set bands and label for classifier
bands = ['ndvi', 'mndwi', 'evi', 'lswi']
label = 'landcover'

In [52]:
sample = image_select.select(bands).sampleRegions(**{
    'collection': tr_classNames,
    'properties': [label],
    'scale': 30
})

In [53]:
# Adds a column of deterministic pseudorandom numbers. 
random.seed(2020)
sample = sample.randomColumn()
split = 0.7

In [54]:
# get training data which is 0.7 of sample
training = sample.filter(ee.Filter.lt('random', split))

In [55]:
# get validation/test data which is 0.3 of sample
validation = sample.filter(ee.Filter.gte('random', split))

In [56]:
# classify training data by randomForest
# img_classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

In [57]:
# Run the classification
classified = image_select.select(bands).classify(classifier)

In [58]:
# Display classification
cl_vis_params = {
    'min': 1, 
    'max': 7, 
    'palette': ['#98ff00','#0b4a8b','#ffc82d','#00ffff','#bf04c2','#ff0000','#008800']
}

legend_keys = ['Mangrove', 'Shrimp', 'Residence', 'Water','Agriculture','Other','Evergreen']
legend_colors = ['#98ff00','#0b4a8b','#ffc82d','#00ffff','#bf04c2','#ff0000','#008800']

Map.centerObject(red_river_delta, 10)
Map.addLayer(classified, cl_vis_params, 'classification')
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')
Map

Map(bottom=116090.0, center=[20.50075230416886, 106.47941624308001], controls=(WidgetControl(options=['positio…

Accuracy assessment

In [59]:
# Training dataset
train_accuracy = classifier.confusionMatrix()

In [60]:
train_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 353, 0, 0, 0, 6, 0, 2],
 [0, 1, 136, 0, 4, 5, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 4, 0, 57, 1, 0, 1],
 [0, 11, 0, 0, 1, 184, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 2, 0, 66]]

In [61]:
train_accuracy.accuracy().getInfo()

0.9544364508393285

In [62]:
train_accuracy.kappa().getInfo()

0.9360977527750052

In [63]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [0.9778393351800554],
 [0.9315068493150684],
 [0],
 [0.9047619047619048],
 [0.9387755102040817],
 [0],
 [0.9705882352941176]]

In [64]:
train_accuracy.consumersAccuracy().getInfo()

[[0,
  0.9671232876712329,
  0.9714285714285714,
  0,
  0.9193548387096774,
  0.9292929292929293,
  0,
  0.9565217391304348]]

Validation dataset

In [65]:
validated = validation.classify(classifier)

In [66]:
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '1_1_1_1_1_1_0_0',
 'properties': {'classification': 2,
  'evi': 1.25413782726452,
  'landcover': 5,
  'lswi': 0.13169817626476288,
  'mndwi': -0.23237250745296478,
  'ndvi': 0.29890620708465576,
  'random': 0.7020675135716052}}

In [67]:
test_accuracy = validated.errorMatrix('landcover', 'classification')

In [68]:
test_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 151, 1, 0, 2, 7, 0, 2],
 [0, 4, 54, 0, 5, 12, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 2, 10, 0, 11, 4, 0, 1],
 [0, 20, 2, 0, 1, 72, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 3, 0, 0, 0, 5, 0, 28]]

In [69]:
test_accuracy.accuracy().getInfo()

0.7959697732997482

In [70]:
test_accuracy.kappa().getInfo()

0.7131298172994577

In [71]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [0.9263803680981595],
 [0.72],
 [0],
 [0.39285714285714285],
 [0.7578947368421053],
 [0],
 [0.7777777777777778]]

In [72]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  0.8388888888888889,
  0.8059701492537313,
  0,
  0.5789473684210527,
  0.72,
  0,
  0.9032258064516129]]

In [74]:
import csv
import os

out_dir = os.path.join(os.path.expanduser('D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/'), 'Results')
training_csv = os.path.join(out_dir, 'train_accuracy.csv')
testing_csv = os.path.join(out_dir, 'test_accuracy.csv')

with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(train_accuracy.getInfo())
    
with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(test_accuracy.getInfo())

Reclassify land cover map

In [75]:
# landcover = landcover.remap(new_class_values, raw_class_values).select(['remapped'], ['classification'])

In [76]:
# landcover = landcover.set('classification_class_values', raw_class_values)
# landcover = landcover.set('classification_class_palette', class_palette)

In [77]:
# Map.addLayer(landcover, {}, 'Final land cover')
# Map

Export the result

In [80]:
import os
out_dir = os.path.join(os.path.expanduser('D:/@MASTER THESIS/@Programming/@Jupyter/Learning/GEE/Thuy/'), 'Results')
out_file = os.path.join(out_dir, 'landcover.tif')

In [81]:
# Export the result directly to your computer:
geemap.ee_export_image(classified, filename=out_file, scale=900)

Generating URL ...
Please wait ...
Data downloaded to D:\@MASTER THESIS\@Programming\@Jupyter\Learning\GEE\Thuy\Results\landcover.tif


In [ ]:
Export the result to Google Drive:

In [ ]:
# geemap.ee_export_image_to_drive(landcover, description='landcover', folder='export', scale=900)